In [1]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16, ResNet50, InceptionV3, MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np
import matplotlib.pyplot as plt

# Define your data directories
train_dir = 'COP_banknote/ds3/Train'
test_dir = 'COP_banknote/ds3/Test'
validation_dir = 'COP_banknote/ds3/Validation'

# Common Hyperparameters
batch_size = 32
num_epochs = 3
num_classes = 12

# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Define a function to build and train a model
def build_and_train_model(base_model, model_name, train_dir, validation_dir, test_dir, input_shape):
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    # Freeze the layers from the base model
    for layer in base_model.layers:
        layer.trainable = False

    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

    print(f"Training {model_name}...")

    train_generator = create_data_generator(train_dir, train_datagen, input_shape)
    validation_generator = create_data_generator(validation_dir, ImageDataGenerator(rescale=1./255), input_shape)
    test_generator = create_data_generator(test_dir, ImageDataGenerator(rescale=1./255), input_shape)

    history = model.fit(
        train_generator,
        steps_per_epoch=len(train_generator),
        validation_data=validation_generator,
        validation_steps=len(validation_generator),
        epochs=num_epochs
    )

    # Evaluate on test data
    test_loss, test_acc = model.evaluate(test_generator, steps=len(test_generator))
    print(f"{model_name} Test accuracy: {test_acc * 100:.2f}%")

    return model, test_generator, history

def create_data_generator(data_dir, data_generator, input_shape):
    return data_generator.flow_from_directory(
        data_dir,
        target_size=input_shape,
        batch_size=batch_size,
        class_mode='categorical',
        classes=['100k', '100kf', '10k', '10kf', '50k', '50kf', '5k', '5kf', '20k', '20kf', '2k', '2kf']
    )

# VGG16
vgg16_base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
vgg16_model, vgg16_test_generator, vgg16_history = build_and_train_model(vgg16_base_model, "VGG16", train_dir, validation_dir, test_dir, (224, 224))

# ResNet50
resnet50_base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
resnet50_model, resnet50_test_generator, resnet50_history = build_and_train_model(resnet50_base_model, "ResNet50", train_dir, validation_dir, test_dir, (224, 224))

# InceptionV3
inceptionv3_base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
inceptionv3_model, inceptionv3_test_generator, inceptionv3_history = build_and_train_model(inceptionv3_base_model, "InceptionV3", train_dir, validation_dir, test_dir, (299, 299))

# MobileNetV2
mobilenetv2_base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
mobilenetv2_model, mobilenetv2_test_generator, mobilenetv2_history = build_and_train_model(mobilenetv2_base_model, "MobileNetV2", train_dir, validation_dir, test_dir, (224, 224))

# Calculate metrics for each model
def calculate_metrics(model, model_name, test_generator):
    y_test = test_generator.classes
    Y_pred = model.predict(test_generator)
    Y_pred_classes = np.argmax(Y_pred, axis=1)

    # Calculate confusion matrix
    cm = confusion_matrix(y_test, Y_pred_classes)

    # Generate classification report
    report = classification_report(y_test, Y_pred_classes, target_names=test_generator.class_indices.keys())

    print(f"Metrics for {model_name}:")
    print("Confusion Matrix:")
    print(cm)
    print("\nClassification Report:")
    print(report)

# Calculate metrics for each model
calculate_metrics(vgg16_model, "VGG16", vgg16_test_generator)
calculate_metrics(resnet50_model, "ResNet50", resnet50_test_generator)
calculate_metrics(inceptionv3_model, "InceptionV3", inceptionv3_test_generator)
calculate_metrics(mobilenetv2_model, "MobileNetV2", mobilenetv2_test_generator)


2023-12-16 23:56:31.692565: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-16 23:56:31.711393: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-16 23:56:31.711409: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-16 23:56:31.711427: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-16 23:56:31.715577: I tensorflow/core/platform/cpu_feature_g

Training VGG16...
Found 0 images belonging to 12 classes.
Found 0 images belonging to 12 classes.
Found 0 images belonging to 12 classes.


ValueError: Unexpected value for `steps_per_epoch`. Received value is 0. Please check the docstring for `model.fit()` for supported values.

In [ ]:
vgg16_model.summary()

In [ ]:
resnet50_model.summary()

In [ ]:
inceptionv3_model.summary()

In [ ]:
mobilenetv2_model.summary()

In [ ]:
import matplotlib.pyplot as plt
def plot_training_metrics(history, model_name):
    plt.figure(figsize=(12, 6))

    # Plot training & validation loss values
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'{model_name} Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    # Plot training & validation accuracy values
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(f'{model_name} Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.show()
plot_training_metrics(vgg16_history, "VGG16")
plot_training_metrics(resnet50_history, "ResNet50")
plot_training_metrics(inceptionv3_history, "InceptionV3")
plot_training_metrics(mobilenetv2_history, "MobileNetV2")

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import roc_auc_score
import numpy as np

# Define a function to plot micro-average ROC curve for a model
def plot_micro_roc_curve(model, model_name, test_generator):
    y_test = test_generator.classes
    Y_pred = model.predict(test_generator)

    # Binarize the labels
    lb = LabelBinarizer()
    y_test_bin = lb.fit_transform(y_test)

    # Calculate micro-average ROC and AUC
    fpr, tpr, _ = roc_curve(y_test_bin.ravel(), Y_pred.ravel())
    roc_auc = auc(fpr, tpr)

    plt.figure()
    lw = 2
    plt.plot(fpr, tpr, color='darkorange', lw=lw, label=f'ROC curve (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'Micro-average ROC Curve for {model_name}')
    plt.legend(loc='lower right')
    plt.show()

# Plot micro-average ROC curve for each model
plot_micro_roc_curve(vgg16_model, "VGG16", vgg16_test_generator)
plot_micro_roc_curve(resnet50_model, "ResNet50", resnet50_test_generator)
plot_micro_roc_curve(inceptionv3_model, "InceptionV3", inceptionv3_test_generator)
plot_micro_roc_curve(mobilenetv2_model, "MobileNetV2", mobilenetv2_test_generator)


In [ ]:
from sklearn.metrics import average_precision_score
import numpy as np

def calculate_mAP(model, test_generator, model_name):
    y_true = test_generator.classes
    Y_pred = model.predict(test_generator)
    
    # Calculate the Average Precision (AP) for each class
    APs = []
    for class_idx in range(num_classes):
        y_true_class = (y_true == class_idx)
        y_pred_class = Y_pred[:, class_idx]
        ap = average_precision_score(y_true_class, y_pred_class)
        APs.append(ap)
    
    # Calculate the Mean Average Precision (mAP)
    mAP = np.mean(APs)
    
    print(f"Mean Average Precision (mAP) for {model_name}: {mAP:.4f}")

# Calculate mAP for each model
calculate_mAP(vgg16_model, vgg16_test_generator, "VGG16")
calculate_mAP(resnet50_model, resnet50_test_generator, "ResNet50")
calculate_mAP(inceptionv3_model, inceptionv3_test_generator, "InceptionV3")
calculate_mAP(mobilenetv2_model, mobilenetv2_test_generator, "MobileNetV2")


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def visualize_predictions(model, model_name, test_generator, num_samples=5):
    # Get a batch of test data and their true labels
    test_images, true_labels = next(test_generator)

    # Get model predictions for the batch
    predicted_labels = model.predict(test_images)

    # Get class labels and indices
    class_labels = test_generator.class_indices
    class_indices = {v: k for k, v in class_labels.items()}

    # Plot the images and their predictions
    plt.figure(figsize=(15, 10))
    for i in range(num_samples):
        plt.subplot(1, num_samples, i + 1)
        plt.imshow(test_images[i])
        true_label = class_indices[np.argmax(true_labels[i])]
        predicted_label = class_indices[np.argmax(predicted_labels[i])]
        plt.title(f'True: {true_label}\nPredicted: {predicted_label}')
        plt.axis('off')

    plt.suptitle(f'{model_name} - Visual Inspection')
    plt.show()

# Visual inspection for VGG16
visualize_predictions(vgg16_model, "VGG16", vgg16_test_generator)

# Visual inspection for ResNet50
visualize_predictions(resnet50_model, "ResNet50", resnet50_test_generator)

# Visual inspection for InceptionV3
visualize_predictions(inceptionv3_model, "InceptionV3", inceptionv3_test_generator)

# Visual inspection for MobileNetV2
visualize_predictions(mobilenetv2_model, "MobileNetV2", mobilenetv2_test_generator)


In [ ]:
#feature maps
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.preprocessing import image

# Load the image
image_path = '914.jpg'
img = image.load_img(image_path, target_size=(224, 224))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
img = preprocess_input(img)

# Define a function to visualize feature maps
def visualize_feature_maps(model, img, layer_names, max_images=3):
    layer_outputs = [layer.output for layer in model.layers if layer.name in layer_names]
    activation_model = Model(inputs=model.input, outputs=layer_outputs)
    intermediate_activations = activation_model.predict(img)
    
    for layer_name, layer_activation in zip(layer_names, intermediate_activations):
        n_features = layer_activation.shape[-1]
        n_features = min(n_features, max_images)
        size = layer_activation.shape[1]
        n_cols = n_features // max_images
        display_grid = np.zeros((size * n_cols, max_images * size))
        
        for col in range(n_cols):
            for row in range(max_images):
                channel_index = col * max_images + row
                if channel_index < n_features:
                    channel_image = layer_activation[0, :, :, channel_index]
                    channel_image -= channel_image.mean()
                    channel_image /= (channel_image.std() + 1e-5)
                    channel_image *= 64
                    channel_image += 128
                    channel_image = np.clip(channel_image, 0, 255).astype('uint8')
                    display_grid[col * size : (col + 1) * size, row * size : (row + 1) * size] = channel_image
        
        scale = 1. / size
        plt.figure(figsize=(scale * display_grid.shape[1], scale * display_grid.shape[0]))
        plt.axis('off')
        plt.title(f'{layer_name} Feature Maps')
        plt.grid(False)
        plt.imshow(display_grid, aspect='auto', cmap='viridis')
        plt.show()

# Define a function to visualize ConvNet filters
def visualize_convnet_filters(model, layer_names, max_filters=3):
    for layer_name in layer_names:
        layer = model.get_layer(layer_name)
        if layer and len(layer.get_weights()) > 0:
            weights = layer.get_weights()[0]
            filter_count = min(weights.shape[3], max_filters)
        
            plt.figure(figsize=(10, 5))
            for i in range(filter_count):
                plt.subplot(1, filter_count, i+1)
                plt.imshow(weights[:, :, 0, i], cmap='viridis')
                plt.axis('off')
                plt.title(f'{layer_name} Filter {i+1}')
            plt.show()
        else:
            print(f"No weights found for layer: {layer_name}")

# Define a function to visualize class output (prediction)
def visualize_class_output(model, img, top_k=3):
    predictions = model.predict(img)
    decoded_predictions = decode_predictions(predictions, top=top_k)[0]
    
    labels = [label for (_, label, prob) in decoded_predictions]
    probabilities = [prob for (_, label, prob) in decoded_predictions]
    
    plt.figure(figsize=(10, 5))
    plt.barh(labels, probabilities)
    plt.xlabel('Probability')
    plt.title('Top Predicted Classes')
    plt.gca().invert_yaxis()
    plt.show()

# Load models
vgg16_model = VGG16(weights='imagenet', include_top=True)
resnet50_model = ResNet50(weights='imagenet', include_top=True)
inceptionv3_model = InceptionV3(weights='imagenet', include_top=True)
mobilenetv2_model = MobileNetV2(weights='imagenet', include_top=True)

# Define layer names to visualize
vgg16_layer_names = ['block1_conv2', 'block2_conv2', 'block3_conv3']
resnet50_layer_names = ['conv1_relu', 'conv2_block3_3_conv', 'conv3_block4_6_conv']
inceptionv3_layer_names = ['mixed0', 'mixed1', 'mixed2']
mobilenetv2_layer_names = ['block_1_project_BN', 'block_3_project_BN', 'block_16_project_BN']

# Visualize feature maps, ConvNet filters, and class output for each model
visualize_feature_maps(vgg16_model, img, vgg16_layer_names)
visualize_convnet_filters(vgg16_model, vgg16_layer_names)
visualize_class_output(vgg16_model, img)


"""this will be used only if VGG16 is not the best model


visualize_feature_maps(inceptionv3_model, img, inceptionv3_layer_names)
visualize_convnet_filters(inceptionv3_model, inceptionv3_layer)


"""

In [ ]:
visualize_feature_maps(resnet50_model, img, resnet50_layer_names)

visualize_class_output(resnet50_model, img)
visualize_convnet_filters(resnet50_model, resnet50_layer_names)

"convnet_filters not working----- visualize_convnet_filters(resnet50_model, resnet50_layer_names)"

In [ ]:
visualize_feature_maps(inceptionv3_model, img, inceptionv3_layer_names)
visualize_convnet_filters(inceptionv3_model, inceptionv3_layer)
visualize_class_output(inceptionv3_model, img)

In [ ]:
visualize_feature_maps(vgg16_model, img, vgg16_layer_names)
visualize_convnet_filters(vgg16_model, vgg16_layer_names)
visualize_class_output(vgg16_model, img)